In [9]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np
# Store excel file locations to variables

Com_2012 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\9. S5 Land\2012_com_s10.csv"
Com_2013 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\9. S5 Land\2013_s4_focus.csv"
Com_2014 = r"C:\Users\warra\Desktop\Freelance\data\data\CommunityMerge\1. Merging by Parts\9. S5 Land\2014_s6_community.csv"

# Read excel files 
df_2012 = pd.read_csv(Com_2012)
df_2013 = pd.read_csv(Com_2013)
df_2014 = pd.read_csv(Com_2014)



In [10]:
#This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.

# Rename columns in df to df_2012 column names
df_2012.rename(columns={
    'LAND_TYPE': 's6_land_type',
    'S10Q1': 's6_q1',
    'S10Q2': 's6_q2',
    'S10Q3': 'Land_PPLD',
    'S10Q4': 'Land_PPLD_Prblm',
    'Round': 'Survey_Round',
    'community': 'Community',
    'fid': 'FID',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)

# Rename columns in df to df_2013 column names
df_2013.rename(columns={
    'land_id': 's6_sr',
    'land_type': 's6_land_type',
    's4_q1': 's6_q1',
    's4_q2': 's6_q2',
    's4_q3': 'Land_KGinMaund'
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [11]:
# Define the column mappings based on the provided positions
# Make dictionaries for each year with updated names
# replace with 'None' where there are no columns
#Here are the updated mapping lists for the given datasets:


mapping_2012 = [
    'cid',
    None,
    's6_land_type',
    's6_q1',
    's6_q2',
    None,
    'Land_PPLD',
    'Land_PPLD_Prblm',
    'Survey_Round',
    'Community',
    'FID',
    'P_ID',
    'D_ID',
    'T_ID',
    'UC_ID',
    'M_ID'
]

mapping_2013 = [
    'cid',
    's6_sr',
    's6_land_type',
    's6_q1',
    's6_q2',
    'Land_KGinMaund',
    None, None, None, None, None, None, None, None, None, None
]


mapping_2014 = [
   'cid',
    's6_sr',
    's6_land_type',
    's6_q1',
    's6_q2', None, None, None, None, None, None, None, None, None, None
]





In [12]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [13]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [14]:
# Usage with dataframes and mappings
dfs = [df_2012, df_2013, df_2014]
mappings = [mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'Survey_Round', 'cid', 'Community', 's6_land_type', 's6_q1', 's6_q2', 'Land_PPLD', 'Land_PPLD_Prblm', 'FID', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column cid
Appending data for column s6_land_type
Appending data for column s6_q1
Appending data for column s6_q2
Appending data for column Land_PPLD
Appending data for column Land_PPLD_Prblm
Appending data for column Survey_Round
Appending data for column Community
Appending data for column FID
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_ID
Appending data for column M_ID
Processing DataFrame with columns: ['Unnamed: 0', 'cid', 's6_sr', 'Land_KGinMaund', 's6_land_type', 's6_q1', 's6_q2']
Appending data for column cid
Appending data for column s6_sr
Appending data for column s6_land_type
Appending data for column s6_q1
Appending data for column s6_q2
Appending data for column Land_KGinMaund
Proc

In [15]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'cid': 'CID',
    's6_sr': 'Land_ID',
    's6_land_type': 'Land_Type',
    's6_q1': 'Land_S_12m',
    's6_q2': 'Land_S_NoP12m'
}


merged_df.rename(columns=rename_mapping, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('7. merged_S6_Land.csv', index=False)

